https://github.com/facebookarchive/python-instagram

https://github.com/LevPasha/Instagram-API-python/blob/master/InstagramAPI/InstagramAPI.py


In [ ]:
#!pip install --user InstagramAPI

In [1]:
from InstagramAPI import InstagramAPI
import time
from datetime import datetime
import db_reacao

Fail to import moviepy. Need only for Video upload.


In [2]:
api = InstagramAPI("vitor.ghiotti@gmail.com", "Deb1Mis2@@")
if (api.login()):
    print("")
else:
    print("Can't login!")

Login success!




# PESQUISA PARA ENTENDER USERID, HASHTAGS E LOCATIONS

In [3]:
query = {}
api.fbUserSearch("vila parque boliche") 
query = api.LastJson

In [4]:
print(query.keys())
print("Users".center(100,"*"))
print(query.get("users"))
print("places".center(100,"*"))
print(query.get("places"))
print("hashtags".center(100,"*"))
print(query.get("hashtags"))
print("has_more".center(100,"*"))
print(query.get("has_more"))

dict_keys(['users', 'places', 'hashtags', 'has_more', 'rank_token', 'clear_client_cache', 'status'])
***********************************************Users************************************************
[{'position': 0, 'user': {'pk': 2970178180, 'username': 'vilaparqueboliche', 'full_name': 'Vila Parque Boliche', 'is_private': False, 'profile_pic_url': 'https://scontent-gru2-2.cdninstagram.com/vp/3416c15939b8a3e9baa715e60ad8775c/5B8EC95A/t51.2885-19/s150x150/14547564_176532216142261_8279261512783101952_a.jpg', 'profile_pic_id': '1385053629437929308_2970178180', 'friendship_status': {'following': False, 'is_private': False, 'incoming_request': False, 'outgoing_request': False, 'is_bestie': False}, 'is_verified': False, 'has_anonymous_profile_picture': False, 'follower_count': 3418, 'byline': '3418 followers', 'mutual_followers_count': 0.0}}, {'position': 3, 'user': {'pk': 3258667867, 'username': 'vilaparqueboliche7945', 'full_name': 'vilaparqueboliche', 'is_private': True, 'profile_pic_

# DEFINIÇÃO DE FUNÇÕES

In [3]:
def get_comments_by_usernameId(usernameId):
    more_available = True
    max_id = ''
    userFeed = {}
    
    api.getUserFeed(usernameId=usernameId,maxid=max_id)
    userFeed = api.LastJson

    while more_available:
        time.sleep(1)
        for item_feed in range(0,userFeed["num_results"]):
            if userFeed["items"][item_feed]["comment_count"]!=0:
                if userFeed["items"][item_feed]["caption"]!=None:
                    if datetime.fromtimestamp(userFeed["items"][item_feed]["caption"]["created_at"])>=query_start_collect:
                        #print(userFeed["items"][itens]["pk"])
                        #print(userFeed["items"][itens]["comment_count"])
                        #print(userFeed["items"][itens]["has_more_comments"])
                        #print(userFeed["items"][itens]["caption"]["text"])
                        #print(datetime.fromtimestamp(userFeed["items"][itens]["caption"]["created_at"]))
                        #print(userFeed["items"][itens]["caption"]["media_id"])
                        print("Registro a ser incluido".center(110,"*"))
                        print("User Feed '" + usernameId + "': " +str(userFeed["items"][item_feed]["caption"]["media_id"]))
                        insert_comments(str(userFeed["items"][item_feed]["caption"]["media_id"]), "UserFeed")
                    else:
                        more_available = False
    
        if more_available:
            more_available = userFeed.get('more_available', False)
            if userFeed.get('more_available', False)==True:
                max_id = userFeed["next_max_id"]   
                api.getUserFeed(usernameId=usernameId,maxid=max_id)
                userFeed = api.LastJson
                time.sleep(2)

In [4]:
def get_comments_by_hashTags(hashTags):
    global usernameId
    
    more_available = True
    max_id = ''
    hTags = {}

    for tags in range(0,len(hashTags)):
        api.getHashtagFeed(hashtagString=hashTags[tags],maxid=max_id)
        hTags = api.LastJson

        while more_available:
            time.sleep(1)
            for item_htag in range(0,len(hTags['items'])):
                if hTags['items'][item_htag]["comment_count"]!=0:
                    if hTags["items"][item_htag]["caption"]!=None:
                        if datetime.fromtimestamp(hTags["items"][item_htag]["caption"]["created_at"])>=query_start_collect:
                            print("Registro a ser incluido".center(110,"*"))
                            print("HarshTag '" + hashTags[tags] + "': " +str(hTags["items"][item_htag]["caption"]["media_id"]))
                            insert_comments(str(hTags["items"][item_htag]["caption"]["media_id"]), "HashTag")
                        else:
                            more_available = False

            if more_available:   
                more_available = hTags.get('more_available', False)
                if hTags.get('more_available', False)==True:
                    max_id = hTags["next_max_id"]   
                    api.getHashtagFeed(hashtagString=hashTags[tags],maxid=max_id)
                    hTags = api.LastJson
                    time.sleep(2)

In [5]:
def get_comments_by_location(listLocations):
    global usernameId
    
    more_available = True
    max_id = ''
    place = {}

    for index in range(0,len(listLocations)):
        api.getLocationFeed(locationId=listLocations[index],maxid=max_id)
        place = api.LastJson

        while more_available:
            time.sleep(1)
            for item_place in range(0,len(place['items'])):
                if place['items'][item_place]["comment_count"]!=0:
                    if place["items"][item_place]["caption"]!=None:
                        if datetime.fromtimestamp(place["items"][item_place]["caption"]["created_at"])>=query_start_collect:
                            print("Registro a ser incluido".center(110,"*"))
                            print("Location '" + listLocations[index] + "': " +str(place["items"][item_place]["caption"]["media_id"]))
                            insert_comments(str(place["items"][item_place]["caption"]["media_id"]), "Place")
                        else:
                            more_available = False

            if more_available:   
                more_available = place.get('more_available', False)
                if place.get('more_available', False)==True:
                    max_id = place["next_max_id"]   
                    api.getLocationFeed(locationId=listLocations[index],maxid=max_id)
                    place = api.LastJson
                    time.sleep(2)

In [6]:
def insert_comments(media_id, origem):
    global erros_insert_comment
    global insert_comment
    global usernameId
    
    dic={'Place':6,'HashTag':5,'UserFeed':4}
    has_more_comments = True
    max_id = ''
    comments = []
    comm={}

    while has_more_comments:
        time.sleep(1)
        api.getMediaComments(media_id, max_id=max_id)
        comm = api.LastJson

        for item_comm in range(0,len(comm['comments'])):
            if datetime.fromtimestamp(comm["comments"][item_comm]["created_at"])>=query_time_ini and datetime.fromtimestamp(comm["comments"][item_comm]["created_at"])<=query_time_end:
                if str(comm["comments"][item_comm]["user_id"])!=usernameId:
                    try:           
                        db_reacao.add_reacao_insta(clientId,dic.get(origem),comm["comments"][item_comm]["text"],0,str(datetime.fromtimestamp(comm["comments"][item_comm]["created_at"])),str(comm["comments"][item_comm]["pk"]))
                        print("Comentário incluido com SUCESSO!")
                        insert_comment +=1
                    except Exception as e:
                        #print("Erro inclusão comentário".center(110,"*"))
                        print("Origem: " + origem)
                        print("Identificador: " + str(comm["comments"][item_comm]["pk"]))
                        print("Comentário: " + str(comm["comments"][item_comm]["text"]))
                        print("User id: " + str(usernameId))
                        print("Media ID: " + str(media_id))
                        print("ERRO: " + str(e))
                        erros_insert_comment +=1
                else:
                    #print("Erro inclusão comentário".center(110,"*"))
                    print("Origem: " + origem)
                    print("Identificador: " + str(comm["comments"][item_comm]["pk"]))
                    print("Comentário: " + str(comm["comments"][item_comm]["text"]))
                    print("User id: " + str(usernameId))
                    print("Media ID: " + str(media_id))
                    print("ERRO: Usuario do comentário é o dono do perfil!")
                    erros_insert_comment +=1
            else:
                #print("Erro inclusão comentário".center(110,"*"))
                print("Origem: " + origem)
                print("Identificador: " + str(comm["comments"][item_comm]["pk"]))
                print("Comentário: " + str(comm["comments"][item_comm]["text"]))
                print("User id: " + str(usernameId))
                print("Media ID: " + str(media_id))
                print("ERRO: Data do comentário(" + str(datetime.fromtimestamp(comm["comments"][item_comm]["created_at"])) + ") fora do intervalo: " + str(query_time_ini) + " e " + str(query_time_end) + "!")
                erros_insert_comment +=1
        has_more_comments = comm.get('has_more_comments', False)
        if comm.get('has_more_comments', False)==True:
            max_id = comm.get('next_max_id', '')
            time.sleep(2)


# INICIALIZAÇÃO DE VARIAVEIS

In [7]:
query_time_ini = datetime.strptime("2018-05-01", '%Y-%m-%d')
query_time_end = datetime.strptime("2018-05-08", '%Y-%m-%d')
query_start_collect = datetime.strptime("2018-05-01", '%Y-%m-%d')
erros_insert_comment = 0
insert_comment = 0
clientId = '8'
usernameId = "2970178180"
listHashTags = ["vilaparqueboliche","vilaparqueboliche💃"]
listLocations = ["1018007885"]
get_comments_by_usernameId(usernameId)
get_comments_by_hashTags(listHashTags)
get_comments_by_location(listLocations)
print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

*******************************************Registro a ser incluido********************************************
User Feed '2970178180': 1774343324511860714
Comentário incluido com SUCESSO!
Origem: UserFeed
Identificador: 17911085515175983
Comentário: @lorena_amorim vamos trabalhar com pratos especiais no cardapio.. o boliche será promocional por 39,90 a hora da pista..
User id: 2970178180
Media ID: 1774343324511860714
ERRO: Usuario do comentário é o dono do perfil!
Origem: UserFeed
Identificador: 17916748630148303
Comentário: Vila COMPRE SEGUIDORES NO WWW. COMPRANDOSEGUIDORES .COM
User id: 2970178180
Media ID: 1774343324511860714
ERRO: Data do comentário(2018-05-08 17:44:03) fora do intervalo: 2018-05-01 00:00:00 e 2018-05-08 00:00:00!
Origem: UserFeed
Identificador: 17910841330169355
Comentário: E a noite tem alguma promoção...?
User id: 2970178180
Media ID: 1774343324511860714
ERRO: Data do comentário(2018-05-08 18:27:31) fora do intervalo: 2018-05-01 00:00:00 e 2018-05-08 00:00:00!
*

In [ ]:
query_time_ini = datetime.strptime("2018-05-01", '%Y-%m-%d')
query_time_end = datetime.strptime("2018-05-08", '%Y-%m-%d')
query_start_collect = datetime.strptime("2018-05-01", '%Y-%m-%d')
erros_insert_comment = 0
insert_comment = 0
clientId = '7'
usernameId = "1940469493"
listHashTags = ["globoformula","globofórmula","globofórmulavv"]
listLocations = ["483953211",'301392450037677']
get_comments_by_usernameId(usernameId)
get_comments_by_hashTags(listHashTags)
get_comments_by_location(listLocations)
print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

In [7]:
query_time_ini = datetime.strptime("2018-05-01", '%Y-%m-%d')
query_time_end = datetime.strptime("2018-05-08", '%Y-%m-%d')
query_start_collect = datetime.strptime("2018-05-01", '%Y-%m-%d')
erros_insert_comment = 0
insert_comment = 0
clientId = '1'
usernameId = "1229681384"
listHashTags = ["casariobotequim"]
listLocations = ["257354853"]
get_comments_by_usernameId(usernameId)
get_comments_by_hashTags(listHashTags)
get_comments_by_location(listLocations)
print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

*******************************************Registro a ser incluido********************************************
User Feed '1229681384': 1773253781771646814
Origem: UserFeed
Identificador: 17884470985206508
Comentário: Cheers 🍻
User id: 1229681384
Media ID: 1773253781771646814
ERRO: Data do comentário(2018-05-06 09:07:27) fora do intervalo: 2018-05-01 00:00:00 e 2018-05-06 00:00:00!
Origem: UserFeed
Identificador: 17853624736254046
Comentário: 👏👏
User id: 1229681384
Media ID: 1773253781771646814
ERRO: Data do comentário(2018-05-06 09:07:45) fora do intervalo: 2018-05-01 00:00:00 e 2018-05-06 00:00:00!
Origem: UserFeed
Identificador: 17882804611215618
Comentário: Olha aí @liamarciad @loja_marcia_zanotti @dulcecolombo96 @dmmiranda1957 @luzimarsouzarosa @suely7754
User id: 1229681384
Media ID: 1773253781771646814
ERRO: Data do comentário(2018-05-06 09:23:54) fora do intervalo: 2018-05-01 00:00:00 e 2018-05-06 00:00:00!
*******************************************Registro a ser incluido******

In [8]:
query_time_ini = datetime.strptime("2018-05-01", '%Y-%m-%d')
query_time_end = datetime.strptime("2018-05-08", '%Y-%m-%d')
query_start_collect = datetime.strptime("2018-05-01", '%Y-%m-%d')
erros_insert_comment = 0
insert_comment = 0
clientId = '4'
usernameId = "6940723892" #6940186779
listHashTags = ["casariobeach","casariobeach‼️👌"]
listLocations = ["727433897454346"]
get_comments_by_usernameId(usernameId)
get_comments_by_hashTags(listHashTags)
get_comments_by_location(listLocations)
print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

*******************************************Resumo do processamento********************************************
Total de inserções de comentario: 0
Erros de inserção de comentario: 0


In [9]:
query_time_ini = datetime.strptime("2018-05-01", '%Y-%m-%d')
query_time_end = datetime.strptime("2018-05-08", '%Y-%m-%d')
query_start_collect = datetime.strptime("2018-05-01", '%Y-%m-%d')
erros_insert_comment = 0
insert_comment = 0
clientId = '6'
usernameId = ""
listHashTags = ["linhaverdevitoria","linhaverdeinteligente"]
listLocations = [""]
#get_comments_by_usernameId(usernameId)
get_comments_by_hashTags(listHashTags)
#get_comments_by_location(listLocations)
print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

*******************************************Resumo do processamento********************************************
Total de inserções de comentario: 0
Erros de inserção de comentario: 0


In [10]:
query_time_ini = datetime.strptime("2018-05-01", '%Y-%m-%d')
query_time_end = datetime.strptime("2018-05-08", '%Y-%m-%d')
query_start_collect = datetime.strptime("2018-05-01", '%Y-%m-%d')
erros_insert_comment = 0
insert_comment = 0
clientId = '2'
usernameId = "4305609001"
listHashTags = ["oleodealegria","oleodealegriavv","óleodealegriavv"]
listLocations = ["1236088003152831"]
get_comments_by_usernameId(usernameId)
get_comments_by_hashTags(listHashTags)
get_comments_by_location(listLocations)
print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

*******************************************Registro a ser incluido********************************************
User Feed '4305609001': 1774273087108093955
Comentário incluido com SUCESSO!
*******************************************Registro a ser incluido********************************************
User Feed '4305609001': 1774272313628042407
Comentário incluido com SUCESSO!
*******************************************Registro a ser incluido********************************************
User Feed '4305609001': 1773514074003149535
Comentário incluido com SUCESSO!
*******************************************Registro a ser incluido********************************************
User Feed '4305609001': 1772838714261400170
Origem: UserFeed
Identificador: 17870736091228681
Comentário: @keyse_jadyvsky amiga já manda Robinho escolher o dele 😂
User id: 4305609001
Media ID: 1772838714261400170
ERRO: Object Reacao[new:13] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.i

# INICIO DA COLETA

In [ ]:
erros_insert_comment = 0
insert_comment=0
get_comments_by_usernameId(usernameId)
get_comments_by_hashTags(listHashTags)
get_comments_by_location(listLocations)
print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))